In [3]:
def get_target(target_type):
	with open(conf_file, "r") as stream:
		try:
			conf_data = yaml.safe_load(stream)
			return conf_data[target_type]
		except yaml.YAMLError as exc:
			return "errordir"
	return "errordir"

In [16]:
#!/usr/bin/env python
# coding: utf-8

from sklearn import metrics as mt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from time import time
import datetime as dt
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import tensorflow as tf
import warnings
import xgboost as xgb
import yaml
#import phdcommon as conf




data_file = "/Users/ravis/Downloads/reduceddata/data/extdata/extmaster_urlstats.csv"
df = pd.read_csv(data_file) # read in the csv file

#Visualizing the hist of data to check normality of independent variable
df_X = df.drop(['status'],axis=1)
#https://www.projectpro.io/recipes/drop-out-highly-correlated-features-in-python

corr_df = pd.DataFrame(df_X.corr().abs())

# Multi Colliniarity analysis on Independent variables 
upper_tri = corr_df.where(np.triu(np.ones(corr_df.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.99)]






X = df.drop(['status', 'url'],axis=1)

s = X.select_dtypes(include='object').columns
X[s] = X[s].astype("float")


y = df['status']


# We did normalize the attributes using StandardScaler() to scale them between 0 and 1 before running models.
# Normalize the data
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

#Direct train/test split
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, y, test_size=0.20, stratify=y, random_state=1234
)
#Create Cross Validation Procedure
cv = StratifiedKFold(n_splits=10, random_state=1234, shuffle=True)

# ### Model Building
# ##### Helper Function
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)
    #fmt = '.2f''d'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# #### Neural Network Model
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.2, random_state=1234)

url_model = tf.keras.Model()
layer_zero = tf.keras.Input(shape=(x_train.shape[1],))
layer1 = tf.keras.layers.Dense(256, activation='relu')(layer_zero)
layer1_bn = tf.keras.layers.BatchNormalization()(layer1)
layer1_do = tf.keras.layers.Dropout(0.2)(layer1_bn)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1_do)
layer2_bn = tf.keras.layers.BatchNormalization()(layer2)
layer2_do = tf.keras.layers.Dropout(0.2)(layer2_bn)
layer3 = tf.keras.layers.Dense(2048, activation='relu')(layer2_do)
layer3_do = tf.keras.layers.Dropout(0.2)(layer3)
layer4 = tf.keras.layers.Dense(128, activation='relu')(layer3_do)
layer4_do = tf.keras.layers.Dropout(0.2)(layer4)
layer5 = tf.keras.layers.Dense(1, activation='sigmoid')(layer4_do)

url_model = tf.keras.Model(inputs=layer_zero, outputs=layer5)
url_model.compile(optimizer=Adam(), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
safety = EarlyStopping(monitor='val_loss', patience=4, min_delta=2e-4)
nn_history = url_model.fit(x_train, y_train, 
                 validation_data=(x_val, y_val),
                 callbacks=[safety],
                 epochs=200, batch_size=25)


target_dir = get_target("modeldir")
model_folder = f"{target_dir}/URLNN"
url_model.save(model_folder)


/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_12303/3457470606.py:44: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = corr_df.where(np.triu(np.ones(corr_df.shape),k=1).astype(np.bool))


ValueError: could not convert string to float: 'length_url'

In [7]:
df_what =df.drop(['status', 'url'],axis=1)

In [10]:
np.unique(df_what['length_url'])

array(['100', '101', '102', '103', '104', '105', '106', '107', '108',
       '109', '110', '111', '112', '113', '114', '115', '116', '117',
       '118', '119', '12', '120', '121', '122', '123', '124', '125',
       '126', '127', '128', '129', '13', '130', '131', '132', '133',
       '134', '135', '136', '1362', '137', '138', '139', '14', '140',
       '141', '142', '143', '144', '145', '146', '147', '148', '149',
       '15', '150', '151', '152', '153', '154', '155', '156', '157',
       '158', '159', '16', '160', '161', '1614', '162', '163', '164',
       '1641', '165', '166', '167', '168', '169', '17', '170', '171',
       '172', '173', '174', '175', '176', '177', '178', '179', '18',
       '180', '181', '182', '183', '184', '185', '186', '187', '188',
       '189', '19', '190', '191', '192', '194', '195', '196', '197',
       '198', '199', '20', '201', '202', '203', '204', '205', '206',
       '207', '208', '209', '21', '210', '211', '212', '213', '214',
       '215', '216', '217',

In [14]:
df_what.dtypes

length_url            object
length_hostname       object
ip                    object
nb_dots               object
nb_hyphens            object
nb_at                 object
nb_qm                 object
nb_and                object
nb_or                 object
nb_eq                 object
nb_underscore         object
nb_tilde              object
nb_percent            object
nb_slash              object
nb_star               object
nb_colon              object
nb_comma              object
nb_semicolumn         object
nb_dollar             object
nb_space              object
nb_www                object
nb_com                object
nb_dslash             object
http_in_path          object
https_token           object
ratio_digits_url      object
ratio_digits_host     object
punycode              object
port                  object
tld_in_path           object
tld_in_subdomain      object
abnormal_subdomain    object
nb_subdomains         object
prefix_suffix         object
shortening_ser

In [18]:
y = df['status']

df = df.drop(['url'],axis=1)

s = df.select_dtypes(include='object').columns
df[s] = df[s].astype("float")

X = df
y = df['status']


# We did normalize the attributes using StandardScaler() to scale them between 0 and 1 before running models.
# Normalize the data
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

ValueError: could not convert string to float: 'length_url'

In [22]:
df.astype({'length_url': 'float'}).dtypes

ValueError: could not convert string to float: 'length_url'